
Exercises and solutions.

In [ ]:

# exercise number 4 from module 5

import matplotlib.pyplot as plt
from ray.rllib.algorithms.ppo import PPOConfig
from ____ import ____

# suppress warnings
import ray
import logging
ray.init(log_to_driver=False, ignore_reinit_error=True, logging_level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

ppo_config = (
    PPOConfig()
    .framework("torch")
    .rollouts(create_env_on_local_worker=True)
    .debugging(seed=0, log_level="ERROR")
    .training(model={"fcnet_hiddens" : [32, 32]})
    .environment(env="Taxi-v3")
)

ppo = ppo_config.build()

ppo_rewards = []
for i in range(10):
    out = ppo.train()
    ppo_rewards.append(out["episode_reward_mean"])
    

dqn_config = (
    ____()
    .framework("torch")
    .rollouts(create_env_on_local_worker=True)
    .debugging(seed=0, log_level="ERROR")
    .training(model={"fcnet_hiddens" : [32, 32]})
    .environment(env="Taxi-v3")
)

dqn = dqn_config.build()

dqn_rewards = []
for i in range(10):
    out = ____.train()
    dqn_rewards.append(out["episode_reward_mean"])
    
plt.plot(ppo_rewards, label="PPO")
plt.plot(dqn_rewards, label="DQN")
plt.legend()
plt.xlabel("training iterations")
plt.ylabel("reward")

In [ ]:

# solution for exercise number 4 from module 5

import numpy as np
import matplotlib.pyplot as plt
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.dqn import DQNConfig

# suppress warnings
import ray
import logging
ray.init(log_to_driver=False, ignore_reinit_error=True, logging_level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

ppo_config = (
    PPOConfig()
    .framework("torch")
    .rollouts(create_env_on_local_worker=True)
    .debugging(seed=0, log_level="ERROR")
    .training(model={"fcnet_hiddens" : [32, 32]})
    .environment(env="Taxi-v3")
)

ppo = ppo_config.build()

ppo_rewards = []
for i in range(10):
    out = ppo.train()
    ppo_rewards.append(out["episode_reward_mean"])
    

dqn_config = (
    DQNConfig()
    .framework("torch")
    .rollouts(create_env_on_local_worker=True)
    .debugging(seed=0, log_level="ERROR")
    .training(model={"fcnet_hiddens" : [32, 32]})
    .environment(env="Taxi-v3")
)

dqn = dqn_config.build()

dqn_rewards = []
for i in range(10):
    out = dqn.train()
    dqn_rewards.append(out["episode_reward_mean"])
    
plt.plot(ppo_rewards, label="PPO")
plt.plot(dqn_rewards, label="DQN")
plt.legend()
plt.xlabel("training iterations")
plt.ylabel("reward")

In [ ]:

# exercise number 9 from module 5

from envs import BasicRecommenderWithHistory
from ray.rllib.algorithms.ppo import PPOConfig
from ray import tune

env_config = {
    "num_candidates" : 2,
    "alpha"          : 0.5,
    "seed"           : 42
}

ppo_config = (
    PPOConfig()\
    .framework("torch")\
    .rollouts(create_env_on_local_worker=True)\
    .debugging(seed=0, log_level="ERROR")\
    .training(model={"fcnet_hiddens" : [64, 64]}, 
              lr=____)\
    .environment(env_config=env_config, env=BasicRecommenderWithHistory)
)

analysis = tune.____(
    "PPO",
    config            = ppo_config.to_dict(),
    ____              = {"training_iteration" : 10},
    checkpoint_freq   = 1,
    verbose           = 0,
    metric            = "episode_reward_mean",
    mode              = "max"
)

____.results_df[["lr", "episode_reward_mean"]]

In [ ]:

# solution for exercise number 9 from module 5

from envs import BasicRecommenderWithHistory
from ray.rllib.algorithms.ppo import PPOConfig
from ray import tune

env_config = {
    "num_candidates" : 2,
    "alpha"          : 0.5,
    "seed"           : 42
}

ppo_config = (
    PPOConfig()\
    .framework("torch")\
    .rollouts(create_env_on_local_worker=True)\
    .debugging(seed=0, log_level="ERROR")\
    .training(model={"fcnet_hiddens" : [64, 64]}, 
              lr=tune.grid_search([1e-2, 1e-3, 1e-4]))\
    .environment(env_config=env_config, env=BasicRecommenderWithHistory)
)

analysis = tune.run(
    "PPO",
    config            = ppo_config.to_dict(),
    stop              = {"training_iteration" : 10},
    checkpoint_freq   = 1,
    verbose           = 0,
    metric            = "episode_reward_mean",
    mode              = "max"
)

analysis.results_df[["config/lr", "episode_reward_mean"]]

In [ ]:

# exercise number 13 from module 5

from ray.rllib.algorithms.ppo import PPOConfig
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

ppo_config_many = (
    PPOConfig()\
    .framework("torch")\
    .____(____)\
    .training(model={"fcnet_hiddens" : [32,32]})
)

ppo_config_single = (
    PPOConfig()\
    .framework("torch")\
    .____(num_rollout_workers=1, num_envs_per_worker=1)\
    .training(model={"fcnet_hiddens" : [32,32]})
)

ppo_many = ppo_config_many.build(env="FrozenLake-v1")
t = time.time()
for i in range(5):
    ppo_many.train()
print(f"Elapsed time with 2 workers, 2 envs each: {time.time()-t:.1f}s.")
ppo_many.stop()

ppo_single = ppo_config_single.build(env="FrozenLake-v1")
t = time.time()
for i in range(5):
    ppo_single.train()
print(f"Elapsed time with 1 worker, 1 env: {time.time()-t:.1f}s.")
ppo_single.stop()

In [ ]:

# solution for exercise number 13 from module 5

from ray.rllib.algorithms.ppo import PPOConfig
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

ppo_config_many = (
    PPOConfig()\
    .framework("torch")\
    .rollouts(num_rollout_workers=2, num_envs_per_worker=2)\
    .training(model={"fcnet_hiddens" : [32,32]})
)

ppo_config_single = (
    PPOConfig()\
    .framework("torch")\
    .rollouts(num_rollout_workers=1, num_envs_per_worker=1)\
    .training(model={"fcnet_hiddens" : [32,32]})
)

ppo_many = ppo_config_many.build(env="FrozenLake-v1")
t = time.time()
for i in range(5):
    ppo_many.train()
print(f"Elapsed time with 2 workers, 2 envs each: {time.time()-t:.1f}s.")
ppo_many.stop()

ppo_single = ppo_config_single.build(env="FrozenLake-v1")
t = time.time()
for i in range(5):
    ppo_single.train()
print(f"Elapsed time with 1 worker, 1 env: {time.time()-t:.1f}s.")
ppo_single.stop()